In [ ]:
import os
import sys
from nectarpy import NectarClient

In [ ]:
API_SECRET = "0xfec3f6491a1889f080ef1fdc048027779645fcb3e2051592915cde62876d46fe"

In [ ]:
nectar_client = NectarClient(API_SECRET,'localhost')

In [ ]:
def main_min_func(list_of_partial_result):
    import pandas as pd
    all_data = pd.concat(list_of_partial_result, ignore_index=True)

    if all_data.empty:
        return {"min_heart_rate": None, "min_age": None, "total_count": 0}
    return {
        "min_heart_rate": int(all_data["heart_rate"].min()),
        "min_age": int(all_data["age"].min()),
        "total_count": int(len(all_data))
    }

In [ ]:
def main_max_func(list_of_partial_result):
    import pandas as pd
    all_data = pd.concat(list_of_partial_result, ignore_index=True)

    if all_data.empty:
        return {"max_heart_rate": None, "max_age": None, "total_count": 0}

    return {
        "max_heart_rate": int(all_data["heart_rate"].max()),
        "max_age": int(all_data["age"].max()),
        "total_count": int(len(all_data))
    }

In [ ]:
def main_compute_node_mean_func(list_of_partial_result):
    import pandas as pd
    all_data = pd.concat(list_of_partial_result, ignore_index=True)
    total_count = len(all_data)
    heart_rate_mean = all_data['heart_rate'].mean() if total_count else 0.0

    if total_count == 0:
        return {"count": 0, "mean": 0.0}

    return {
        "count": int(total_count),
        "mean": float(heart_rate_mean),
    }

In [ ]:
def main_compute_node_variance_func(list_of_partial_result):
    import pandas as pd

    # Gộp tất cả raw DataFrame từ các worker
    all_data = pd.concat(list_of_partial_result, ignore_index=True)

    total_count = len(all_data)
    if total_count == 0:
        return {"count": 0, "mean": 0.0, "variance": 0.0}

    mean_age = all_data['age'].mean()
    variance_age = all_data['age'].var(ddof=0)  # ddof=0 để tính variance theo công thức chuẩn

    return {
        "count": int(total_count),
        "mean": float(mean_age),
        "variance": float(variance_age),
    }

In [ ]:
def main_compute_node_linear_regression_func(list_of_partial_result):
    import pandas as pd
    from sklearn.linear_model import LinearRegression

    # Gộp tất cả raw DataFrame từ workers
    all_data = pd.concat(list_of_partial_result, ignore_index=True)

    # Lọc dữ liệu và loại bỏ NaN
    df_filtered = all_data[all_data['age'] > 10].dropna(subset=["heart_rate", "age", "height"])

    total_count = len(df_filtered)
    if total_count == 0:
        return {"coef": [], "intercept": 0.0, "count": 0}

    X_train = df_filtered[["heart_rate", "age"]]
    Y_train = df_filtered["height"]

    model = LinearRegression()
    model.fit(X_train, Y_train)

    return {
        "coef": model.coef_.tolist(),
        "intercept": float(model.intercept_),
        "count": int(total_count),
    }

In [ ]:
result = nectar_client.byoc_query(
    main_func = main_min_func, 
    is_separate_data = False,
    bucket_ids=[70279094317368356596175311423469920804414768838518678723060543206733034143809,88093996874795333302344705952553473497540234507805766807785030728783544536135],
    policy_indexes=[0,0]
)

In [ ]:
result

In [ ]:
print(nectar_client.get_result(result))

In [ ]:
### Multiple data node

In [ ]:
print(result)